In [1]:
from mesa.batchrunner import BatchRunner, BatchRunnerMP
from mesa.time import RandomActivation
from model import *
from customizeTime import *
import pickle
import numpy as np
import uuid

from customizeTime import *
import env_settings

TYPE_MAP = {0:'vert',
            1:'horz'}

settings.MUTATION_PRO = 0.01

class A_3(GenAgent):
    def muta_genetic_info(self):
        # no need for mutation
        pass
    
        
class M_3(GenModel):
    def get_agent_config(self):
        return A_3
    
    def init_env(self):
        #no influence
        return [self.env_val + self.env_slope * i for i in range(10000)]
    
    def get_schedule(self):
        return RandomActivationMutationWithConstrain
    #
    def __init__(self, N, width, height, specific_kargs):
        #mut_var, init_var, env_val
        self.model_name = 'M3'
        
        init_ratio = specific_kargs['init_ratio']
        self.env_val = specific_kargs['env_val']
        self.env_slope = specific_kargs['slope']
         
        settings.INIT_MEAN = specific_kargs['gene_mean']
        settings.INIT_SD = specific_kargs['gene_sd']
        settings.INIT_LOW = settings.INIT_MEAN - 1 * 2 * settings.INIT_SD
        settings.INIT_HIGH = settings.INIT_MEAN + 1 * 2 * settings.INIT_SD

        settings.POPU_MUTA_SD = specific_kargs['muta_sd']
        settings.POPU_MUTA_LOW = -2 * settings.POPU_MUTA_SD
        settings.POPU_MUTA_HIGH = 2 * settings.POPU_MUTA_SD
        
        self.specific_kargs_str = '_'.join([str(i) for i in (self.env_val, settings.INIT_SD ,settings.POPU_MUTA_SD)])
        super().__init__(N, width, height, init_ratio)
        

    
    def model_level_record_init(self):
        data_file_name = check_path("./res/")
        data_file_name = check_path(data_file_name + self.model_name)
        data_file_name = check_path(data_file_name + "/" + self.specific_kargs_str)
        data_file_name = data_file_name + '/' + str(uuid.uuid1())
        print('data in file ', data_file_name)
        self.data_file = open(data_file_name, 'w')
        self.data_file.write('#' + '\t'.join(
            ('step',
             TYPE_MAP[0], TYPE_MAP[0] + '_beta', TYPE_MAP[0] + '_gene', 'var',  
             TYPE_MAP[1], TYPE_MAP[1] + '_beta', TYPE_MAP[1] + '_gene', 'var'))+ '\n')
       
    def get_popu_beta(self, gen_type):
        return np.mean([agent.self_gen_cof for agent in self.schedule.agents if agent.gen_type == gen_type])
    
    def get_gene_mean(self, gen_type):
        return [np.mean(agent.gen_info) for agent in self.schedule.agents if agent.gen_type == gen_type]
    
    def get_popu_gene_mean(self, gene_mean):
        return np.mean(gene_mean)

    def get_popu_gene_var(self, gene_mean):
        return np.var(gene_mean)

    def collect_data(self):
        ver_mean = self.get_gene_mean(0)
        hor_mean = self.get_gene_mean(1)
        return (self.schedule.steps,
                self.get_ver_popu_size(), self.get_popu_beta(0), self.get_popu_gene_mean(ver_mean), self.get_popu_gene_var(ver_mean),
                self.get_her_popu_size(), self.get_popu_beta(1), self.get_popu_gene_mean(hor_mean), self.get_popu_gene_var(hor_mean))
        
        
    def step(self):
        
        data = self.collect_data()
        #print(data)
        self.data_file.write('\t'.join([str(d) for d in data]) + '\n')
        if self.schedule.steps % 100 == 0: 
            self.data_file.flush()
        super().step()
        

In [2]:
import sys

#0.01, 0.005, 0.001
sys.argv = 'm3 0.5 0.03'.split(' ')

muta_sd, slope = float(sys.argv[1]), float(sys.argv[2])

fixed_params = {'N':2000, 'width':100, 'height':100}

variable_params = {'specific_kargs': [{'slope': slope,'env_val': 0.5 , 'init_ratio':0.5, 'gene_mean':0.5, 'gene_sd':muta_sd, 'muta_sd':muta_sd}]}

batch_run = BatchRunnerMP(
    M_3, 1,
    variable_parameters=variable_params,
    fixed_parameters=fixed_params,
    iterations=1,
    max_steps=5000,
)
batch_run.run_all()

0it [00:00, ?it/s]

data in file  ./res/M3/0.5_0.5_0.5/58b8e8f2-d444-11e9-a62b-f2189850a77c
data in file  ./res/M3/0.5_0.5_0.5/58b98e38-d444-11e9-9dad-f2189850a77c
data in file  ./res/M3/0.5_0.5_0.5/58ba52d2-d444-11e9-b0d7-f2189850a77c
data in file  ./res/M3/0.5_0.5_0.5/58baf3ea-d444-11e9-996c-f2189850a77c
0 population size:  2000
0 population size:  2000
0 population size:  2000
0 population size:  2000
1 population size:  3393
1 population size:  3417
1 population size:  3468
1 population size:  3434
2 population size:  5287
2 population size:  5276
2 population size:  5333
2 population size:  5330
3 population size:  7148
3 population size:  7154
3 population size:  7210
3 population size:  7205
4 population size:  8503
4 population size:  8481
4 population size:  8543
4 population size:  8532
5 population size:  9228
5 population size:  9244
5 population size:  9238
5 population size:  9277
6 population size:  9500
6 population size:  9517
6 population size:  9536
6 population size:  9564
7 population

Process ForkPoolWorker-3:
Process ForkPoolWorker-2:
Process ForkPoolWorker-4:
Process ForkPoolWorker-1:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/multiprocess/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.7/site-packages/multiprocess/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.7/site-packages/multiprocess/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.7/site-packages/multiprocess/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.7/site-packages/multiprocess/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/site-packages/multiprocess/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/site-

KeyboardInterrupt: 

In [5]:
import sys

#0.01, 0.005, 0.001
sys.argv = 'm3 0.1 0.02'.split(' ')

muta_sd, slope = float(sys.argv[1]), float(sys.argv[2])

fixed_params = {'N':2000, 'width':100, 'height':100}

variable_params = {'specific_kargs': {'slope': slope,'env_val': 0.5 , 'init_ratio':0.5, 'gene_mean':0.5, 'gene_sd':muta_sd, 'muta_sd':muta_sd}}

para = {**variable_params, **fixed_params}
m = M_3(**para)
m.data_file = sys.stdout
while True:
    m.step()

data in file  ./res/M3/0.5_0.1_0.1/1f0901b8-d445-11e9-be85-f2189850a77c
0	961	0.9998155944580608	0.49942927988063884	0.0001840798204846292	1039	0.9998153692065658	0.49957194106710007	0.00018444755898406912
0 population size:  2000
1	1666	0.9997385836694593	0.49945316115570476	0.0002643491316455054	1791	0.9998149475910537	0.49967034655359494	0.00018683132380472805
1 population size:  3457
2	2567	0.9992556203174121	0.49969086287266384	0.00033588633855760836	2787	0.9994091672427395	0.49982601578917146	0.00018525120643373989
2 population size:  5354
3	3448	0.9979545769361676	0.4993691869080531	0.0003954108672608541	3835	0.9982162064721622	0.5000766904979417	0.00018807119636595206
3 population size:  7283
4	4104	0.9959048731091932	0.4993662196530442	0.00042099805118304235	4562	0.9962357527373963	0.5002138784601527	0.00019228167390358602
4 population size:  8666
5	4448	0.9930698089446559	0.4994539819677857	0.00044169026602558276	4941	0.9934550687009034	0.5002991759174032	0.000195969938151200

100 population size:  0
101	0.0	nan	nan	nan	0.0	nan	nan	nan
101 population size:  0
102	0.0	nan	nan	nan	0.0	nan	nan	nan
102 population size:  0
103	0.0	nan	nan	nan	0.0	nan	nan	nan
103 population size:  0
104	0.0	nan	nan	nan	0.0	nan	nan	nan
104 population size:  0
105	0.0	nan	nan	nan	0.0	nan	nan	nan
105 population size:  0
106	0.0	nan	nan	nan	0.0	nan	nan	nan
106 population size:  0
107	0.0	nan	nan	nan	0.0	nan	nan	nan
107 population size:  0
108	0.0	nan	nan	nan	0.0	nan	nan	nan
108 population size:  0
109	0.0	nan	nan	nan	0.0	nan	nan	nan
109 population size:  0
110	0.0	nan	nan	nan	0.0	nan	nan	nan
110 population size:  0
111	0.0	nan	nan	nan	0.0	nan	nan	nan
111 population size:  0
112	0.0	nan	nan	nan	0.0	nan	nan	nan
112 population size:  0
113	0.0	nan	nan	nan	0.0	nan	nan	nan
113 population size:  0
114	0.0	nan	nan	nan	0.0	nan	nan	nan
114 population size:  0
115	0.0	nan	nan	nan	0.0	nan	nan	nan
115 population size:  0
116	0.0	nan	nan	nan	0.0	nan	nan	nan
116 population size:  0
117	0.0	nan	nan	

/usr/local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3157: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


300 population size:  0
301	0.0	nan	nan	nan	0.0	nan	nan	nan
301 population size:  0
302	0.0	nan	nan	nan	0.0	nan	nan	nan
302 population size:  0
303	0.0	nan	nan	nan	0.0	nan	nan	nan
303 population size:  0
304	0.0	nan	nan	nan	0.0	nan	nan	nan
304 population size:  0
305	0.0	nan	nan	nan	0.0	nan	nan	nan
305 population size:  0
306	0.0	nan	nan	nan	0.0	nan	nan	nan
306 population size:  0
307	0.0	nan	nan	nan	0.0	nan	nan	nan
307 population size:  0
308	0.0	nan	nan	nan	0.0	nan	nan	nan
308 population size:  0
309	0.0	nan	nan	nan	0.0	nan	nan	nan
309 population size:  0
310	0.0	nan	nan	nan	0.0	nan	nan	nan
310 population size:  0
311	0.0	nan	nan	nan	0.0	nan	nan	nan
311 population size:  0
312	0.0	nan	nan	nan	0.0	nan	nan	nan
312 population size:  0
313	0.0	nan	nan	nan	0.0	nan	nan	nan
313 population size:  0
314	0.0	nan	nan	nan	0.0	nan	nan	nan
314 population size:  0
315	0.0	nan	nan	nan	0.0	nan	nan	nan
315 population size:  0
316	0.0	nan	nan	nan	0.0	nan	nan	nan
316 population size:  0
317	0.0	nan	nan	

438 population size:  0
439	0.0	nan	nan	nan	0.0	nan	nan	nan
439 population size:  0
440	0.0	nan	nan	nan	0.0	nan	nan	nan
440 population size:  0
441	0.0	nan	nan	nan	0.0	nan	nan	nan
441 population size:  0
442	0.0	nan	nan	nan	0.0	nan	nan	nan
442 population size:  0
443	0.0	nan	nan	nan	0.0	nan	nan	nan
443 population size:  0
444	0.0	nan	nan	nan	0.0	nan	nan	nan
444 population size:  0
445	0.0	nan	nan	nan	0.0	nan	nan	nan
445 population size:  0
446	0.0	nan	nan	nan	0.0	nan	nan	nan
446 population size:  0
447	0.0	nan	nan	nan	0.0	nan	nan	nan
447 population size:  0
448	0.0	nan	nan	nan	0.0	nan	nan	nan
448 population size:  0
449	0.0	nan	nan	nan	0.0	nan	nan	nan
449 population size:  0
450	0.0	nan	nan	nan	0.0	nan	nan	nan
450 population size:  0
451	0.0	nan	nan	nan	0.0	nan	nan	nan
451 population size:  0
452	0.0	nan	nan	nan	0.0	nan	nan	nan
452 population size:  0
453	0.0	nan	nan	nan	0.0	nan	nan	nan
453 population size:  0
454	0.0	nan	nan	nan	0.0	nan	nan	nan
454 population size:  0
455	0.0	nan	nan	

...

9444 population size:  0
9445	0.0	nan	nan	nan	0.0	nan	nan	nan
9445 population size:  0
9446	0.0	nan	nan	nan	0.0	nan	nan	nan
9446 population size:  0
9447	0.0	nan	nan	nan	0.0	nan	nan	nan
9447 population size:  0
9448	0.0	nan	nan	nan	0.0	nan	nan	nan
9448 population size:  0
9449	0.0	nan	nan	nan	0.0	nan	nan	nan
9449 population size:  0
9450	0.0	nan	nan	nan	0.0	nan	nan	nan
9450 population size:  0
9451	0.0	nan	nan	nan	0.0	nan	nan	nan
9451 population size:  0
9452	0.0	nan	nan	nan	0.0	nan	nan	nan
9452 population size:  0
9453	0.0	nan	nan	nan	0.0	nan	nan	nan
9453 population size:  0
9454	0.0	nan	nan	nan	0.0	nan	nan	nan
9454 population size:  0
9455	0.0	nan	nan	nan	0.0	nan	nan	nan
9455 population size:  0
9456	0.0	nan	nan	nan	0.0	nan	nan	nan
9456 population size:  0
9457	0.0	nan	nan	nan	0.0	nan	nan	nan
9457 population size:  0
9458	0.0	nan	nan	nan	0.0	nan	nan	nan
9458 population size:  0
9459	0.0	nan	nan	nan	0.0	nan	nan	nan
9459 population size:  0
9460	0.0	nan	nan	nan	0.0	nan	nan	nan
9460 pop

9600 population size:  0
9601	0.0	nan	nan	nan	0.0	nan	nan	nan
9601 population size:  0
9602	0.0	nan	nan	nan	0.0	nan	nan	nan
9602 population size:  0
9603	0.0	nan	nan	nan	0.0	nan	nan	nan
9603 population size:  0
9604	0.0	nan	nan	nan	0.0	nan	nan	nan
9604 population size:  0
9605	0.0	nan	nan	nan	0.0	nan	nan	nan
9605 population size:  0
9606	0.0	nan	nan	nan	0.0	nan	nan	nan
9606 population size:  0
9607	0.0	nan	nan	nan	0.0	nan	nan	nan
9607 population size:  0
9608	0.0	nan	nan	nan	0.0	nan	nan	nan
9608 population size:  0
9609	0.0	nan	nan	nan	0.0	nan	nan	nan
9609 population size:  0
9610	0.0	nan	nan	nan	0.0	nan	nan	nan
9610 population size:  0
9611	0.0	nan	nan	nan	0.0	nan	nan	nan
9611 population size:  0
9612	0.0	nan	nan	nan	0.0	nan	nan	nan
9612 population size:  0
9613	0.0	nan	nan	nan	0.0	nan	nan	nan
9613 population size:  0
9614	0.0	nan	nan	nan	0.0	nan	nan	nan
9614 population size:  0
9615	0.0	nan	nan	nan	0.0	nan	nan	nan
9615 population size:  0
9616	0.0	nan	nan	nan	0.0	nan	nan	nan
9616 pop

9742 population size:  0
9743	0.0	nan	nan	nan	0.0	nan	nan	nan
9743 population size:  0
9744	0.0	nan	nan	nan	0.0	nan	nan	nan
9744 population size:  0
9745	0.0	nan	nan	nan	0.0	nan	nan	nan
9745 population size:  0
9746	0.0	nan	nan	nan	0.0	nan	nan	nan
9746 population size:  0
9747	0.0	nan	nan	nan	0.0	nan	nan	nan
9747 population size:  0
9748	0.0	nan	nan	nan	0.0	nan	nan	nan
9748 population size:  0
9749	0.0	nan	nan	nan	0.0	nan	nan	nan
9749 population size:  0
9750	0.0	nan	nan	nan	0.0	nan	nan	nan
9750 population size:  0
9751	0.0	nan	nan	nan	0.0	nan	nan	nan
9751 population size:  0
9752	0.0	nan	nan	nan	0.0	nan	nan	nan
9752 population size:  0
9753	0.0	nan	nan	nan	0.0	nan	nan	nan
9753 population size:  0
9754	0.0	nan	nan	nan	0.0	nan	nan	nan
9754 population size:  0
9755	0.0	nan	nan	nan	0.0	nan	nan	nan
9755 population size:  0
9756	0.0	nan	nan	nan	0.0	nan	nan	nan
9756 population size:  0
9757	0.0	nan	nan	nan	0.0	nan	nan	nan
9757 population size:  0
9758	0.0	nan	nan	nan	0.0	nan	nan	nan
9758 pop

9876 population size:  0
9877	0.0	nan	nan	nan	0.0	nan	nan	nan
9877 population size:  0
9878	0.0	nan	nan	nan	0.0	nan	nan	nan
9878 population size:  0
9879	0.0	nan	nan	nan	0.0	nan	nan	nan
9879 population size:  0
9880	0.0	nan	nan	nan	0.0	nan	nan	nan
9880 population size:  0
9881	0.0	nan	nan	nan	0.0	nan	nan	nan
9881 population size:  0
9882	0.0	nan	nan	nan	0.0	nan	nan	nan
9882 population size:  0
9883	0.0	nan	nan	nan	0.0	nan	nan	nan
9883 population size:  0
9884	0.0	nan	nan	nan	0.0	nan	nan	nan
9884 population size:  0
9885	0.0	nan	nan	nan	0.0	nan	nan	nan
9885 population size:  0
9886	0.0	nan	nan	nan	0.0	nan	nan	nan
9886 population size:  0
9887	0.0	nan	nan	nan	0.0	nan	nan	nan
9887 population size:  0
9888	0.0	nan	nan	nan	0.0	nan	nan	nan
9888 population size:  0
9889	0.0	nan	nan	nan	0.0	nan	nan	nan
9889 population size:  0
9890	0.0	nan	nan	nan	0.0	nan	nan	nan
9890 population size:  0
9891	0.0	nan	nan	nan	0.0	nan	nan	nan
9891 population size:  0
9892	0.0	nan	nan	nan	0.0	nan	nan	nan
9892 pop

IndexError: list index out of range